In [1]:
BASE_DIR = "/Users/denis/MyProjects/Summarization"
GRAPH_BASE_DIR = "/Users/denis/MyProjects/uncased_L-12_H-768_A-12"

In [2]:
import os
from bert_experimental.feature_extraction.bert_feature_extractor import BERTFeatureExtractor
from tree_rank_summaraiser import TreeRankSummariser
import global_const
import pandas as pd
import glob
import time
import logging
import numpy as np
logging.basicConfig(level=logging.DEBUG)

In [3]:
GRAPH_PATH = os.path.join(GRAPH_BASE_DIR, "model_for_inference.pbtxt")
VOCAB_PATH = os.path.join(GRAPH_BASE_DIR, "vocab.txt")

encoder = BERTFeatureExtractor(GRAPH_PATH, VOCAB_PATH)

2020-05-10 15:04:13,069 :  From /Users/denis/anaconda3/envs/Summarization/lib/python3.6/site-packages/bert_experimental/feature_extraction/text_preprocessing.py:287: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


2020-05-10 15:04:13,167 :  From /Users/denis/anaconda3/envs/Summarization/lib/python3.6/site-packages/bert_experimental/feature_extraction/bert_feature_extractor.py:52: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.


2020-05-10 15:04:13,169 :  From /Users/denis/anaconda3/envs/Summarization/lib/python3.6/site-packages/bert_experimental/feature_extraction/bert_feature_extractor.py:38: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.


2020-05-10 15:04:13,192 :  Using temporary folder as model directory: /var/folders/lx/02q9j0f52ds3f_h7jnq_dvmm0000gp/T/tmpva9g9ptn
2020-05-10 15:04:13,194 :  Using config: {'_model_dir': '/var/folders/lx/02q9j0f52ds3f_h7jnq_dvmm0000gp/T/tmpva9g9ptn', '_

In [4]:
summariser = TreeRankSummariser(3, encoder)

In [5]:
pattern_str = "/Users/denis/MyProjects/Summarization/data/Companies' news/Norwegian Cruise Line/*"
files_list = glob.glob(pattern_str)
data_list = []
company_name_list = ["Norwegian Cruise Line", "Norwegian Cruise"]
# company_name_list = ["Carnival"]
# company_name_list = ["Pfizer"]

companies = [
    {
        global_const.COMPANY_ISIN: "Norwegian Cruise Line",
        global_const.COMPANY_NAMES: ["Norwegian Cruise Line", "Norwegian Cruise"]
     },
    {
        global_const.COMPANY_ISIN: "Carnival corporation",
        global_const.COMPANY_NAMES: ["carnival corporation", "ccl"]
    }
]

time_list = []
for path in files_list:
    print(path)
    with open(path) as file:
        article_str = file.read()
    
    start = time.time()
    article_sum_dict = summariser.get_summary_for_companies(article_str, companies)
    finish = time.time()
    time_list.append(finish - start)
    for isin, summary in article_sum_dict.items():
        data_list.append({
            "News": article_str,
            "Summarization": summary,
            "isin": isin
        })
    
pd.DataFrame(data=data_list).to_excel(f"/Users/denis/MyProjects/Summarization/data/Companies' news/Norwegian Cruise Line.xls")

DEBUG:tree_rank_summaraiser:Extracted 17 sentences
DEBUG:tree_rank_summaraiser:Pattern: (^|[\.,":'!?()\s])(norwegian cruise line|norwegian cruise)($|[\.,":'!?()\s])
DEBUG:tree_rank_summaraiser:Sentence 0 got weight. ISIN - Norwegian Cruise Line
DEBUG:tree_rank_summaraiser:why carnival corporation, royal caribbean, and norwegian cruise line stocks crashed this morning -- then made it all back

what happened
in early trading monday, shares of carnival corporation (nyse:ccl) sank 8.5% and royal caribbean (nyse:rcl) dove 9.2%, while norwegian cruise line holdings (nyse:nclh) fared worse than either of those -- plunging 12.5%.
DEBUG:tree_rank_summaraiser:Sentence 7 got weight. ISIN - Norwegian Cruise Line
DEBUG:tree_rank_summaraiser:despite having "ample evidence to believe their fleet of luxury liners were incubators for the new coronavirus," said the article, carnival, royal caribbean, and norwegian cruise (yes, all three were called out by name -- not a blue-chip cruise line was spared) 

/Users/denis/MyProjects/Summarization/data/Companies' news/Norwegian Cruise Line/Why Carnival Corporation, Royal Caribbean, and Nor
/Users/denis/MyProjects/Summarization/data/Companies' news/Norwegian Cruise Line/NCLH INVESTOR ALERT: Bronstein, Gewirtz & Grossman
/Users/denis/MyProjects/Summarization/data/Companies' news/Norwegian Cruise Line/Nothing Seems to Go Right for Carnival, Royal Cari
/Users/denis/MyProjects/Summarization/data/Companies' news/Norwegian Cruise Line/A Look Into Norwegian Cruise Line's Price Over Ear
/Users/denis/MyProjects/Summarization/data/Companies' news/Norwegian Cruise Line/ROSEN, GLOBAL INVESTOR COUNSEL, Encourages Investo


In [6]:
time_np = np.array(time_list)

In [7]:
time_np.mean()

1.1977033138275146

In [8]:
np.median(time_np)

1.0589218139648438

In [9]:
time_np

array([1.05892181, 0.92600584, 1.52645612, 1.05520892, 1.42192388])